In [1]:
import time
import random
import numpy as np
import pandas as pd
import sys
import pickle
import h5py
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import torch
import learn2learn as l2l
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import StepLR, CosineAnnealingLR
from functions.fftc import fft2c_new as fft2c
from functions.fftc import ifft2c_new as ifft2c
from functions.math import complex_abs, complex_mul, complex_conj
# The corase reconstruction is the rss of the zerofilled multi-coil kspaces
# after inverse FT.
from functions.data.transforms import UnetDataTransform_sens_TTT, center_crop
# Import a torch.utils.data.Dataset class that takes a list of data examples, a path to those examples
# a data transform and outputs a torch dataset.
from functions.data.mri_dataset import SliceDataset
# Unet architecture as nn.Module
from functions.models.unet import Unet
# Function that returns a MaskFunc object either for generatig random or equispaced masks
from functions.data.subsample import create_mask_for_mask_type
# Implementation of SSIMLoss
from functions.training.losses import SSIMLoss
from functions.helper import evaluate2c_imagepair
### after you install bart 0.7.00 from https://mrirecon.github.io/bart/, import it as follows
sys.path.insert(0,'/cheng/bart-0.7.00/python/')
os.environ['TOOLBOX_PATH'] = "/cheng/bart-0.7.00/"
import bart


plt.rcParams.update({"text.usetex": True, "font.family": "serif", "font.serif": ["Computer Modern Roman"]})

colors = ['b','r','k','g','m','c','tab:brown','tab:orange','tab:pink','tab:gray','tab:olive','tab:purple']

markers = ["v","o","^","1","*",">","d","<","s","P","X"]
FONTSIZE = 22

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# seed
SEED = 1
random.seed(SEED)
np.random.seed(SEED)
torch.cuda.manual_seed(SEED)
torch.manual_seed(SEED)


### Load the data

In [2]:
# data path
# path_test = '/cheng/metaMRI/metaMRI/data_dict/E11.1/Q/brain_test_AXT1POST_Skyra_5-8.yaml'
# path_test_sensmaps = '/cheng/metaMRI/metaMRI/data_dict/E11.1/Q/sensmap_test/'
path_test = '/cheng/metaMRI/metaMRI/data_dict/TTT_paper/TTT_brain_test_100.yaml'
path_test_sensmaps = '/cheng/metaMRI/metaMRI/data_dict/TTT_paper/sensmap_brain_test/'

# mask function and data transform
mask_function = create_mask_for_mask_type(mask_type_str = 'random', self_sup = False, 
                    center_fraction = 0.08, acceleration = 4.0, acceleration_total = 3.0)

data_transform = UnetDataTransform_sens_TTT('multicoil', mask_func = mask_function, use_seed=True, mode='adapt')

# training dataset and data loader
testset = SliceDataset(dataset = path_test, path_to_dataset='', 
                path_to_sensmaps = path_test_sensmaps, provide_senmaps=True, 
                challenge="multicoil", transform = data_transform, use_dataset_cache=True)

test_dataloader = torch.utils.data.DataLoader(dataset = testset, batch_size = 1, 
                shuffle = False, generator = torch.Generator().manual_seed(1), pin_memory = False)


In [3]:
checkpoint_path = '/cheng/metaMRI/metaMRI/save/E11.6_joint(l1_1e-5_P)_T300_150epoch/E11.6_joint(l1_1e-5_P)_T300_150epoch_E150_best.pth'
model = Unet(in_chans=2, out_chans=2, chans=32, num_pool_layers=4, drop_prob=0.0)
model.load_state_dict(torch.load(checkpoint_path))
model = model.to(device)

### without TTT

In [4]:
loss_l1_history_=[]
loss_ssim_history_=[]
ssim_fct = SSIMLoss()
l1_loss = torch.nn.L1Loss(reduction='sum')

for iter, batch in tqdm(enumerate(test_dataloader)): 
    input_image, target_image, ground_truth_image, mean, std, fname, slice_num, input_kspace, input_mask, target_kspace, target_mask, sens_maps, binary_background_mask = batch
    input_image = input_image.to(device)
    target_image = target_image.to(device)
    input_kspace = input_kspace.to(device)
    input_mask = input_mask.to(device)
    sens_maps = sens_maps.to(device)
    std = std.to(device)
    mean = mean.to(device)
    ground_truth_image = ground_truth_image.to(device)
    binary_background_mask = binary_background_mask.to(device)

    model_output = model(input_image)
    model_output = model_output * std + mean
    # supervised loss [x, fθ(A†y)]
    # L1
    loss_l1 = (l1_loss(model_output, target_image) / torch.sum(torch.abs(target_image))).item()
    # SSIM = 1 - loss
    output_image_1c = complex_abs(torch.moveaxis(model_output , 1, -1 ))
    # center crop
    min_size = min(ground_truth_image.shape[-2:])
    crop = torch.Size([min_size, min_size])
    ground_truth_image = center_crop( ground_truth_image, crop )
    output_image_1c = center_crop( output_image_1c, crop )
    loss_ssim = 1 - ssim_fct(output_image_1c, ground_truth_image, data_range = ground_truth_image.max().unsqueeze(0)).item()
    #print('Test SSIM loss: ',loss_ssim)

    loss_l1_history_.append(loss_l1)
    loss_ssim_history_.append(loss_ssim)

100it [00:20,  4.88it/s]


In [6]:
print("Testing average L1 loss: ", sum(loss_l1_history_) / len(loss_l1_history_))
print("Testing average SSIM loss: ", sum(loss_ssim_history_) / len(loss_ssim_history_))

Testing average L1 loss:  0.28124732211232184
Testing average SSIM loss:  0.7954672402143479


### Test-time training

In [8]:
best_loss_l1_history=[]
best_loss_l1_index_history=[]
best_loss_ssim_history=[]
best_loss_ssim_index_history=[]
ssim_fct = SSIMLoss()
l1_loss = torch.nn.L1Loss(reduction='sum')
adapt_lr = 0.00001
TTT_epoch = 500

for iter, batch in tqdm(enumerate(test_dataloader)): 
    input_image, target_image, ground_truth_image, mean, std, fname, slice_num, input_kspace, input_mask, target_kspace, target_mask, sens_maps, binary_background_mask = batch
    input_image = input_image.to(device)
    target_image = target_image.to(device)
    input_kspace = input_kspace.to(device)
    input_mask = input_mask.to(device)
    sens_maps = sens_maps.to(device)
    std = std.to(device)
    mean = mean.to(device)
    ground_truth_image = ground_truth_image.to(device)
    binary_background_mask = binary_background_mask.to(device)

    # model re-init
    model.load_state_dict(torch.load(checkpoint_path))

    # each data point TTT
    optimizer = torch.optim.Adam(model.parameters(),lr=adapt_lr)
    
    loss_l1_history = []
    loss_ssim_history = []
    self_loss_history = []

    for iteration in range(TTT_epoch):
        # fθ(A†y)
        model_output = model(input_image)
        model_output = model_output * std + mean
        # supervised loss [x, fθ(A†y)]
        # L1
        loss_l1 = (l1_loss(model_output, target_image) / torch.sum(torch.abs(target_image))).item()
        #print('Test L1 loss: ',loss_l1)
        loss_l1_history.append(loss_l1)
        # SSIM = 1 - loss
        output_image_1c = complex_abs(torch.moveaxis(model_output , 1, -1 ))

        # center crop
        min_size = min(ground_truth_image.shape[-2:])
        crop = torch.Size([min_size, min_size])
        ground_truth_image = center_crop( ground_truth_image, crop )
        output_image_1c = center_crop( output_image_1c, crop )

        loss_ssim = 1 - ssim_fct(output_image_1c, ground_truth_image, data_range = ground_truth_image.max().unsqueeze(0)).item()
        #print('Test SSIM loss: ',loss_ssim)
        loss_ssim_history.append(loss_ssim)

        # self-supervised loss
        # fθ(A†y)
        model_output = torch.moveaxis(model_output, 1, -1 )
        # S fθ(A†y)
        output_sens_image = complex_mul(model_output, sens_maps)
        # FS fθ(A†y)
        Fimg = fft2c(output_sens_image)
        # MFS fθ(A†y) = A fθ(A†y)
        Fimg_forward = Fimg * input_mask
        # consistency loss [y, Afθ(A†y)]
        loss_self = l1_loss(Fimg_forward, input_kspace) / torch.sum(torch.abs(input_kspace))

        optimizer.zero_grad()
        loss_self.backward()
        optimizer.step()
        #train_loss += loss.item()
        #print('TTT loss: ',loss_self.item())
        self_loss_history.append(loss_self.item())



    # 
    best_loss_l1 = min(loss_l1_history)
    print('best L1: ', best_loss_l1)
    best_loss_l1_epoch = np.argmin(loss_l1_history)
    print('best L1 epoch: ', best_loss_l1_epoch)
    best_loss_ssim = max(loss_ssim_history)
    print('best SSIM:', best_loss_ssim)
    best_loss_ssim_epoch = np.argmax(loss_ssim_history)
    print('best SSIM epoch: ', best_loss_ssim_epoch)

    best_loss_l1_index_history.append(best_loss_l1_epoch)
    best_loss_ssim_index_history.append(best_loss_ssim_epoch)
    best_loss_l1_history.append(best_loss_l1)
    best_loss_ssim_history.append(best_loss_ssim)


print("Testing average L1 loss: ", sum(best_loss_l1_history) / len(best_loss_l1_history))
print("Testing average L1 loss epoch: ", sum(best_loss_l1_index_history) / len(best_loss_l1_index_history))
print("Testing average SSIM loss: ", sum(best_loss_ssim_history) / len(best_loss_ssim_history))
print("Testing average SSIM loss epoch: ", sum(best_loss_ssim_index_history) / len(best_loss_ssim_index_history))

0it [00:00, ?it/s]

1it [00:32, 32.56s/it]

best L1:  0.22337999939918518
best L1 epoch:  128
best SSIM: 0.8006564378738403
best SSIM epoch:  144


2it [01:04, 32.40s/it]

best L1:  0.17287781834602356
best L1 epoch:  118
best SSIM: 0.8021988868713379
best SSIM epoch:  125


...

best L1:  0.19737188518047333
best L1 epoch:  80
best SSIM: 0.8729242086410522
best SSIM epoch:  80


99it [1:44:01, 65.85s/it]

best L1:  0.18586452305316925
best L1 epoch:  148
best SSIM: 0.8617737293243408
best SSIM epoch:  148


100it [1:45:06, 63.07s/it]

best L1:  0.26342397928237915
best L1 epoch:  83
best SSIM: 0.7894495725631714
best SSIM epoch:  5
Testing average L1 loss:  0.20899275436997414
Testing average L1 loss epoch:  177.34
Testing average SSIM loss:  0.841446081995964
Testing average SSIM loss epoch:  151.32
